# RecordLinKage

<div align="center">

  
  <a href="https://github.com/1treu1/Deduplicacion-de-Datos/tree/main/RecordLinKage" target="_blank">
    <img width="1024", src="https://recordlinkage.readthedocs.io/en/latest/_images/indexing_basic.png" width="500" height="500"></a>


<br>
  <a href="https://colab.research.google.com/drive/1t6wKMkjDRyG1NvLqMpvxuZdwLfO3nUua?usp=sharing"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  <a href="https://github.com/1treu1/Deduplicacion-de-Datos/tree/main/RecordLinKage"><img src="https://img.shields.io/badge/github-Open In Github-brightgreen.svg" alt="Open In Github"></a>
</br>

</div>

Esta libreria permite cencontrar registros duplicados usando varios algoritmos de similitud semantica. Estos son algunos que maneja la libreria:

- Jaro–Winkler distance: https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance
- Jaro Similarity: https://rosettacode.org/wiki/Jaro_similarity
- Levenstein Distance: https://en.wikipedia.org/wiki/Levenshtein_distance

Problema:
- En Mercado libre tienen una base de datos inmensa en su ERP de SIESA. Entre los datos maestros esta la informacion de los clientes. Debido a malas practicas, hay muchos registros dupicados, que hacen que la calidad de los datos no sea la esperada. 
Se necesita hallar todos los duplicados de la base de datos y dejar un registro unico de los clientes.

Solucion:
- Usar RecordLinkage para entrenar un modelo NLP con el fin de agrupar los registros duplicados en la base de datos


## Paso 1: Instalar Librerias

In [1]:
!pip install recordlinkage
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.1 MB/s eta 0:00:00


# Paso 2: Crear Base de Datos de Ejemplo

Crearemos a 1000 clientes de una empresa, sus atributos son Nombre, Apellido, DNI y Fecha de Nacimiento. Para esto, usaremos la libreria Faker, que permite generar nombres y datos aleatorios de lo que queramos :3

In [5]:
import pandas as pd
from faker import Faker

# Crear una instancia de Faker
fake = Faker("es_ES")
Faker.seed(222)
# Crear una lista vacía para almacenar los usuarios
usuarios = []

# Generar 1000 usuarios aleatorios
for _ in range(1000):
    nombre = fake.first_name()
    apellido = fake.last_name()
    dni = fake.unique.random_number(digits=8)
    fecha_nacimiento = fake.date_of_birth(minimum_age=18, maximum_age=90)
    usuarios.append((nombre, apellido, dni, fecha_nacimiento))

# Crear un dataframe a partir de la lista de usuarios
df = pd.DataFrame(usuarios, columns=['Nombre', 'Apellido', 'DNI', 'Fecha de nacimiento'])

# Mostrar los primeros registros del dataframe
print(df.head())

    Nombre   Apellido       DNI Fecha de nacimiento
0  Melania     Campos  31574101          1971-06-18
1  Mariano     Angulo   3918234          1998-05-16
2  Jacinto    Salgado  99348528          1992-11-17
3   Víctor  Benavides   2180849          1946-09-10
4   Silvio      Belda  65820934          2002-12-17


# Paso 3: Ahora crearemos algunos duplicados:

In [42]:
df1 = df.sample(frac=0.2, random_state=333) #Creando 200 duplicados
df1.index = range(len(df1))
df1.shape

(200, 4)

In [43]:
print("Datos originales", df.shape)
print("Datos duplicados", df1.shape)

Datos originales (1000, 4)
Datos duplicados (200, 4)


# Paso 4: Limpieza
* Ahora vamos a hacer una limpieza de los datos

In [44]:
import recordlinkage

In [45]:
dfA = df.copy()
dfB = df1.copy()

dfA.shape, dfB.shape

((1000, 4), (200, 4))

* Convertimos cada columna en tipo str

In [46]:
dfA['Nombre'] = dfA['Nombre'].astype(str)
dfA['Apellido'] = dfA['Apellido'].astype(str)
dfA['Fecha de nacimiento'] = dfA['Fecha de nacimiento'].astype(str)
dfB['Nombre'] = dfB['Nombre'].astype(str)
dfB['Apellido'] = dfB['Apellido'].astype(str)
dfB['Fecha de nacimiento'] = dfB['Fecha de nacimiento'].astype(str)

* Pasamos todas las cadenas a minusculas

In [47]:
dfA['Nombre'] = dfA['Nombre'].str.lower()
dfA['Apellido'] = dfA['Apellido'].str.lower()
dfB['Nombre'] = dfB['Nombre'].str.lower()
dfB['Apellido'] = dfB['Apellido'].str.lower()

* Eliminamos carecteres especiales

In [48]:
CharEspetial = ["á","é","í","ó","ú","à","è","ì","ò","ù","-","/","  "]
Char = ["a","e","i","o","u","a","e","i","o","u","",""," "]
for i in zip(CharEspetial, Char):
  dfA['Nombre'] = dfA['Nombre'].str.replace(i[0], i[1])
  dfA['Apellido'] = dfA['Apellido'].str.replace(i[0], i[1])
  dfB['Nombre'] = dfB['Nombre'].str.replace(i[0], i[1])
  dfB['Apellido'] = dfB['Apellido'].str.replace(i[0], i[1])

# Paso 5: Hallando los duplicados
* Usaremos la distancia de Jaro-Winkler para encontrar la similitud entre las cadenas

In [49]:
indexer = recordlinkage.Index() #Instanciamos el objeto
indexer.block('Nombre') # Escogemos una columna para hallar las combinaciones posibles
candidate_links = indexer.index(dfA, dfB) # Hallando combinaciones posibles

print("candidate_links:", len(candidate_links))

compare_cl = recordlinkage.Compare() # Objeto que permite comparar los dos sets de datos
# Le decimos como se llaman las columnas que queremos
# Comparar en cada uno de los sets de datos. Y tambien
# Como queremos que se guarde en la salida

compare_cl.exact('Nombre', 'Nombre', label='Nombre')
compare_cl.string('Apellido', 'Apellido', method='jarowinkler', threshold=0., label='Apellido')
compare_cl.string('Fecha de nacimiento', 'Fecha de nacimiento',method='jarowinkler', label='Fecha de nacimiento')
compare_cl.numeric('DNI', 'DNI', label='DNI')


candidate_links: 446


<Compare>

In [50]:
features = compare_cl.compute(candidate_links, dfA, dfB) # Hallando los duplicados
features.head()

,,Nombre,Apellido,Fecha de nacimiento,DNI
0,23,1,1.0,0.801905,0.0
791,23,1,1.0,1.000000,1.0
5,55,1,1.0,1.000000,1.0
6,89,1,1.0,0.901481,0.0
158,89,1,1.0,1.000000,1.0


* El algoritmo nos dice que tanta similitud existe entre las columnas comparadas. Esta similitud va de 0 a 1.
* Si sumamos las filas, nos damos cuenta de que es mas probable de encontrar un duplicado si su resultado es mayor a 3

In [53]:
matches = features[features.sum(axis=1) > 3] # Sumando filas
print(len(matches),"Duplicados")

200 Duplicados


In [54]:
matches # Registros duplicados

,,Nombre,Apellido,Fecha de nacimiento,DNI
791,23,1,1.0,1.0,1.0
5,55,1,1.0,1.0,1.0
158,89,1,1.0,1.0,1.0
8,83,1,1.0,1.0,1.0
9,62,1,1.0,1.0,1.0
...,...,...,...,...,...
945,93,1,1.0,1.0,1.0
957,156,1,1.0,1.0,1.0
962,165,1,1.0,1.0,1.0
970,37,1,1.0,1.0,1.0


* Veamos algunos ejemplos:

In [55]:
dfA.loc[9]

Nombre                      roman
Apellido                  velasco
DNI                      11893770
Fecha de nacimiento    1973-02-21
Name: 9, dtype: object

In [56]:
dfB.loc[62]

Nombre                      roman
Apellido                  velasco
DNI                      11893770
Fecha de nacimiento    1973-02-21
Name: 62, dtype: object

* Podemos ver que efectivamente son duplicados